### connection

In [1]:
from elasticsearch import helpers, Elasticsearch
import csv
import os
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from elasticsearch.helpers import reindex
import pandas as pd
import re
import torch
# requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
# koalpaka 연결
from transformers import pipeline, AutoModelForCausalLM

MODEL = 'beomi/KoAlpaca-Polyglot-5.8B'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline(
    'text-generation', 
    model=model,
    tokenizer=MODEL,
    device=0
)

def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:", 
        do_sample=True, 
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [3]:
# torch 사용가능 확인
print(torch.__version__)
print(torch.cuda.is_available())

2.0.1+cu118
True


In [4]:
os.chdir('..')
os.getcwd()

'c:\\Users\\jieun\\Desktop\\project\\elastic'

In [5]:
# 엘라스틱 연결
es = Elasticsearch('https://172.16.204.158:9200', basic_auth=['jieun','jieun123'], verify_certs=False)
es.info()

c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://172.16.204.158:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'name': 'JIEUN', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'm_kcEMjTSWqeNtfhEZ_sgw', 'version': {'number': '8.9.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': 'a813d015ef1826148d9d389bd1c0d781c6e349f0', 'build_date': '2023-08-10T05:02:32.517455352Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype

---

### 데이터 bulk load

In [146]:
# CSV 파일 불러오기
# data_path = "data/"
# csv_files = os.listdir(data_path)
# dataframes = [pd.read_csv(data_path + file) for file in csv_files]
# print(csv_files)

dd = pd.read_csv("C:/Users/jieun/Desktop/project/elastic/data/3_서울시 종로구 모범음식점 지정 현황.csv")
# dd = dd.fillna(0)
index_name = "서울시_종로구_모범음식점_지정_현황"

field_name = dd.columns.to_list()
f_type = dd.dtypes

# Define a custom analyzer
custom_analyzer = {
    "tokenizer": "my_tokenizer",
    "filter": ["lowercase", "trim", "stop", "my_filter", "nori_filter"],
    "char_filter": ["html_strip"]
}

# Define a custom filter for the analyzer
custom_filter = {
    "type": "pattern_replace",
    "pattern": "[^a-zA-Z0-9가-힣\\s]",
    "replacement": " "
}

# Define a custom tokenizer
custom_tokenizer = {
    "type": "nori_tokenizer",
    "decompound_mode": "mixed",
    "discard_punctuation": False
}

# nori 토큰 필터 설정
nori_filter = {
    "type": "nori_part_of_speech",
    "stoptags": ["E", "IC", "J", "MAG", "MAJ", "MM", "SP", "SSC", "SSO", "SC", "SE", "XPN", "XSA", "XSN", "XSV", "UNA", "NA", "VSV"]
}

# Create an index with custom analyzer and tokenizer settings
index_settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "my_analyzer": custom_analyzer
            },
            "filter": {
                "my_filter": custom_filter,
                "nori_filter": nori_filter
            },
            "tokenizer": {
                "my_tokenizer": custom_tokenizer
            }
        }
    },
    "mappings": {
        "properties": {}  # We'll add the properties dynamically
    }
}

# Iterate through the dataframes and dynamically add properties to the mappings
# for df in dataframes:
fields = dd.columns.tolist()

for field in fields:
    field_type = "text" if pd.api.types.is_string_dtype(dd[field]) else "long"
    if field_type == "text":
        index_settings["mappings"]["properties"][field] = {"type": field_type, "analyzer": "my_analyzer"}
    else:
        index_settings["mappings"]["properties"][field] = {"type": field_type}

# Create the index with the defined settings
es.indices.create(index = index_name, body = index_settings)

# Bulk insert data into Elasticsearch
actions = []
for _, row in dd.iterrows():
    data = row.to_dict()
    actions.extend([{"_index": index_name, "_source": data}])

helpers.bulk(es, actions)


C:\Users\jieun\AppData\Local\Temp\ipykernel_63644\4264031060.py:74: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index = index_name, body = index_settings)
c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


BulkIndexError: 34 document(s) failed to index.

In [136]:
dd

,SUBWAY_ID,STATN_ID,STATN_NM,호선이름
0,1001,1001000100,소요산,1호선
1,1001,1001000101,동두천,1호선
2,1001,1001000102,보산,1호선
3,1001,1001000103,동두천중앙,1호선
4,1001,1001000104,지행,1호선
...,...,...,...,...
655,1093,1093004014,달미,서해선
656,1093,1093004015,선부,서해선
657,1093,1093004016,초지,서해선
658,1093,1093004017,시우,서해선


---

### 엘라스틱 내 인덱스 확인

In [147]:
index_list = es.indices.get_alias().keys()
for index in index_list:
    print(index)

.kibana_8.9.1_001
.internal.alerts-observability.logs.alerts-default-000001
.internal.alerts-observability.uptime.alerts-default-000001
.fleet-file-data-agent-000001
.apm-agent-configuration
.apm-source-map
.internal.alerts-observability.slo.alerts-default-000001
.fleet-files-agent-000001
.kibana_security_session_1
.internal.alerts-observability.apm.alerts-default-000001
.kibana_task_manager_8.9.1_001
실시간도착_역정보
.internal.alerts-observability.metrics.alerts-default-000001
.security-7
서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드
.kibana_ingest_8.9.1_001
.kibana_analytics_8.9.1_001
.apm-custom-link
.kibana_alerting_cases_8.9.1_001
.kibana_security_solution_8.9.1_001
서울시_종로구_모범음식점_지정_현황
.security-profile-8
.internal.alerts-security.alerts-default-000001
.async-search


c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\jieun\AppData\Local\Temp\ipykernel_63644\3956420855.py:1: ElasticsearchWarning: this request accesses system indices: [.kibana_8.9.1_001, .apm-agent-configuration, .kibana_security_session_1, .kibana_task_manager_8.9.1_001, .security-7, .kibana_ingest_8.9.1_001, .kibana_analytics_8.9.1_001, .apm-custom-link, .kibana_alerting_cases_8.9.1_001, .kibana_security_solution_8.9.1_001, .security-profile-8, .async-search], but in a future major version, direct access to system indices will be prevented by default
  index_list = es.indices.get_alias().keys()


### 매핑정보 확인

In [120]:
# 매핑 정보 가져오기
mapping = es.indices.get_mapping(index="서울시_종로구_모범음식점_지정_현황")

# 필드와 해당 데이터 타입 확인
for field, field_info in mapping[index_name]['mappings']['properties'].items():
    data_type = field_info['type']
    print(f"Field: {field}, Data Type: {data_type}")

Field: 급수시설구분, Data Type: text
Field: 소재지도로명, Data Type: text
Field: 소재지전화번호, Data Type: text
Field: 소재지지번, Data Type: text
Field: 시군구코드, Data Type: long
Field: 신청일자, Data Type: long
Field: 업소명, Data Type: text
Field: 업태명, Data Type: text
Field: 영업장면적(㎡), Data Type: long
Field: 주된음식, Data Type: text
Field: 지정년도, Data Type: long
Field: 지정번호, Data Type: long
Field: 지정일자, Data Type: long
Field: 행정동명, Data Type: text
Field: 허가(신고)번호, Data Type: text


c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### 토크나이저 정보 확인

In [121]:
# 인덱스의 매핑 설정 가져오기
mapping = es.indices.get_mapping(index=index_name)

# 분석 설정 확인
analyzer_settings = mapping[index_name]['mappings']['properties']['소재지도로명']['analyzer']
print("Analyzer Settings:", analyzer_settings)

Analyzer Settings: my_analyzer


c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [36]:
# 정규 표현식 패턴
korean_pattern = re.compile("[가-힣]")

# 리스트 컴프리헨션을 사용하여 한글이 포함된 요소 찾기
korean_elements = [item for item in index_list if korean_pattern.search(item)]

# 결과 출력
print(korean_elements)

['실시간도착역정보', '서울시_지하철_호선별_첫차와_막차_정보전철역코드외부코드', '서울시_종로구_모범음식점_지정_현황']


### 인덱스 제거

In [143]:
# 삭제할 인덱스 이름 지정
index_name = "실시간도착_역정보"  # 삭제하려는 인덱스 이름을 설정하세요

# 인덱스 삭제 명령 실행
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"인덱스 '{index_name}' 삭제 완료")
else:
    print(f"인덱스 '{index_name}'는 이미 존재하지 않습니다.")

인덱스 '실시간도착_역정보' 삭제 완료


c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### 토크나이저 제거

In [ ]:
# 제거할 토크나이저 이름 지정
tokenizer_name_to_remove = "nori_sample"  # 제거하려는 토크나이저 이름을 설정하세요
index_name = "my-index2"  # 인덱스 이름을 적절히 변경하세요

# 인덱스를 닫습니다.
es.indices.close(index=index_name)

# 토크나이저를 포함한 설정에서 토크나이저를 제거합니다.
tokenizer_settings = {
    "settings": {
        "analysis": {
            "tokenizer": {
                tokenizer_name_to_remove: None  # 토크나이저를 제거하려면 None으로 설정합니다.
            }
        }
    }
}

# 설정 업데이트
es.indices.put_settings(index=index_name, body=tokenizer_settings)

# 인덱스를 다시 엽니다.
es.indices.open(index=index_name)

---

## match all

In [98]:
# 검색할 쿼리 설정 (예: 모든 문서 검색)
query = {
    "query": {
        "match_all": {}
    }
}

# 인덱스에서 데이터 검색
search_results = es.search(index='서울시_종로구_모범음식점_지정_현황', body=query)

# 검색 결과 출력
for hit in search_results['hits']['hits']:
    print(hit['_source'])

{'시군구코드': 3000000, '지정년도': 2018, '지정번호': 43, '신청일자': 20171031, '지정일자': 20180119, '업소명': '신안촌', '소재지도로명': '서울특별시 종로구 사직로12길 8, (내자동,,153,154-1,155-1 (1층))', '소재지지번': '서울특별시 종로구 내자동  152번지  ,153,154-1,155-1 (1층)', '허가(신고)번호': '3000000-101-1986-06323', '업태명': '분식', '주된음식': '낙지, 홍어', '영업장면적(㎡)': 99.98, '행정동명': '사직동', '급수시설구분': '상수도전용', '소재지전화번호': '02 725 7744'}
{'시군구코드': 3000000, '지정년도': 2018, '지정번호': 26, '신청일자': 20171031, '지정일자': 20180119, '업소명': '만가', '소재지도로명': '서울특별시 종로구 사직로 103-19, (필운동,(지상1층))', '소재지지번': '서울특별시 종로구 필운동  206번지  (지상1층)', '허가(신고)번호': '3000000-101-1991-06583', '업태명': '한식', '주된음식': '장어, 생등심', '영업장면적(㎡)': 110.0, '행정동명': '사직동', '급수시설구분': '상수도전용', '소재지전화번호': '02720 5797'}
{'시군구코드': 3000000, '지정년도': 2016, '지정번호': 144, '신청일자': 20161004, '지정일자': 20161205, '업소명': '백제고기나라', '소재지도로명': '서울특별시 종로구 돈화문로11길 41, (낙원동,2층)', '소재지지번': '서울특별시 종로구 낙원동  136번지 0호  2층', '허가(신고)번호': '3000000-101-1974-05722', '업태명': '일식', '주된음식': '초밥', '영업장면적(㎡)': 167.05, '행정동명': '종로1.2.3.4가동', '급수시설구분': '상수도전용',

C:\Users\jieun\AppData\Local\Temp\ipykernel_63644\1692958983.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  search_results = es.search(index='서울시_종로구_모범음식점_지정_현황', body=query)
c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### just 토큰화

In [148]:
analysis_request = {
    "text": '서울특별시 종로구에 있는 맛있는 쌈밥집인데 정말 맛있다구'
}

analysis_result = es.indices.analyze(index=index_name, body=analysis_request)

# 분석 결과 출력
tokens = [token['token'] for token in analysis_result['tokens']]
print("Nori 토큰화 결과:", " / ".join(tokens))
# print(analysis_result['tokens'])


Nori 토큰화 결과: 서울특별시 / 종로구에 / 있는 / 맛있는 / 쌈밥집인데 / 정말 / 맛있다구


C:\Users\jieun\AppData\Local\Temp\ipykernel_63644\753945645.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  analysis_result = es.indices.analyze(index=index_name, body=analysis_request)
c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## 쿼리문
### multi match

In [6]:
from elasticsearch_dsl import Search, Q

# 모든 필드에서 일치하는 문서 검색
query = Q("multi_match", query="종로", fields=["*"])
print(query)

# 검색 질의 생성
s = Search(using=es, index=['*'])
s = s.query(query)
s = s.extra(size=20)

# 결과 가져오기
response = s.execute()

# 검색 결과 출력
k = 0
for hit in response:
    k = k + 1
    print(hit.meta.index, hit.meta.score, hit.to_dict())
print(k)

MultiMatch(fields=['*'], query='종로')
실시간도착_역정보 4.635003 {'SUBWAY_ID': 1001, 'STATN_ID': 1001000129, 'STATN_NM': '종로5가', '호선이름': '1호선'}
실시간도착_역정보 4.635003 {'SUBWAY_ID': 1001, 'STATN_ID': 1001000130, 'STATN_NM': '종로3가', '호선이름': '1호선'}
실시간도착_역정보 4.635003 {'SUBWAY_ID': 1003, 'STATN_ID': 1003000329, 'STATN_NM': '종로3가', '호선이름': '3호선'}
실시간도착_역정보 4.635003 {'SUBWAY_ID': 1005, 'STATN_ID': 1005000534, 'STATN_NM': '종로3가', '호선이름': '5호선'}
서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드 4.276704 {'호선': '05호선', '상/하행선': 2, '요일': 3, '전철역코드': 2535, '외부코드': '534', '전철역명': '종로3가', '첫차시간': 54730, '첫차출발역코드': 2531, '첫차도착역코드': 2561, '첫차출발역명': '애오개', '첫차도착역명': '마천', '막차시간': 235430, '막차출발역코드': 2511, '막차도착역코드': 2541, '막차출발역명': '방화', '막차도착역명': '왕십리'}
서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드 4.276704 {'호선': '05호선', '상/하행선': 1, '요일': 2, '전철역코드': 2535, '외부코드': '534', '전철역명': '종로3가', '첫차시간': 54120, '첫차출발역코드': 2541, '첫차도착역코드': 2511, '첫차출발역명': '왕십리', '첫차도착역명': '방화', '막차시간': 235630, '막차출발역코드': 2561, '막차도착역코드': 2531, '막차출발역명': '마천', '막차도착역명': 

c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.204.158'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [7]:
response

<Response: [<Hit(실시간도착_역정보/0ipCfooBCuFcPWlmZ-r_): {'SUBWAY_ID': 1001, 'STATN_ID': 1001000129, 'STATN_NM': '종로5...}>, <Hit(실시간도착_역정보/0ypCfooBCuFcPWlmZ-r_): {'SUBWAY_ID': 1001, 'STATN_ID': 1001000130, 'STATN_NM': '종로3...}>, <Hit(실시간도착_역정보/XypCfooBCuFcPWlmaOsA): {'SUBWAY_ID': 1003, 'STATN_ID': 1003000329, 'STATN_NM': '종로3...}>, <Hit(실시간도착_역정보/vypCfooBCuFcPWlmaOsA): {'SUBWAY_ID': 1005, 'STATN_ID': 1005000534, 'STATN_NM': '종로3...}>, <Hit(서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드/PSpCfooBCuFcPWlm3O6u): {'호선': '05호선', '상/하행선': 2, '요일': 3, '전철역코드': 2535, '외부코드': '...}>, <Hit(서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드/ZypCfooBCuFcPWlm3O6u): {'호선': '05호선', '상/하행선': 1, '요일': 2, '전철역코드': 2535, '외부코드': '...}>, <Hit(서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드/gSpCfooBCuFcPWlm3O6u): {'호선': '03호선', '상/하행선': 1, '요일': 3, '전철역코드': 319, '외부코드': '3...}>, <Hit(서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드/kCpCfooBCuFcPWlm3O6u): {'호선': '01호선', '상/하행선': 1, '요일': 1, '전철역코드': 153, '외부코드': '1...}>, <Hit(서울시_지하철_호선별_첫차와_막차_정보_전철역코드_외부코드/oCpCfooBCuFcPWlm3O

In [8]:
q = f"{response}정보를 요약 후 종로구에 대해 설명해줘"
ask(q)

c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

---

## LLM

In [9]:
!nvidia-smi

Sun Sep 10 20:15:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro P1000                 WDDM  | 00000000:73:00.0  On |                  N/A |
| 37%   49C    P8              N/A /  N/A |   3771MiB /  4096MiB |     23%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [24]:
!pip install -q -U bitsandbytes

In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})